# Dynamic FAISS Sentence RAG Demo

- FAISS index for vector similarity search
- Runtime insertion of new sentences
- Paragraph query returns top-N most relevant memory sentences


In [1]:
%pip install -q faiss-cpu sentence-transformers numpy


[notice] A new release of pip is available: 25.2 -> 26.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import re
from dataclasses import dataclass
from typing import List

import faiss
import numpy as np
from sentence_transformers import SentenceTransformer


/Users/jakemckenna/Code/dungeon-masters-companion/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
SENTENCE_SPLIT_RE = re.compile(r"(?<=[.!?])\s+|\n+")

def split_into_sentences(text: str) -> List[str]:
    parts = [p.strip() for p in SENTENCE_SPLIT_RE.split(text or "") if p.strip()]
    return parts

@dataclass
class SearchHit:
    sentence: str
    score: float

class DynamicSentenceMemory:
    def __init__(self, model_name: str = "sentence-transformers/all-MiniLM-L6-v2"):
        self.model = SentenceTransformer(model_name)
        self.index = None
        self.sentences: List[str] = []

    def _embed(self, texts: List[str]) -> np.ndarray:
        vectors = self.model.encode(
            texts,
            convert_to_numpy=True,
            normalize_embeddings=True,
        ).astype(np.float32)
        return vectors

    def add_sentences(self, sentences: List[str]) -> None:
        clean = [s.strip() for s in sentences if s and s.strip()]
        if not clean:
            return

        vectors = self._embed(clean)
        if self.index is None:
            self.index = faiss.IndexFlatIP(vectors.shape[1])
        self.index.add(vectors)
        self.sentences.extend(clean)

    def add_text(self, text: str) -> None:
        self.add_sentences(split_into_sentences(text))

    def search(self, paragraph: str, top_n: int = 3) -> List[SearchHit]:
        if self.index is None or not self.sentences:
            return []

        query = self._embed([paragraph])
        k = min(top_n, len(self.sentences))
        scores, idxs = self.index.search(query, k)

        hits: List[SearchHit] = []
        for score, idx in zip(scores[0], idxs[0]):
            if idx < 0:
                continue
            hits.append(SearchHit(sentence=self.sentences[int(idx)], score=float(score)))
        return hits


In [4]:
memory = DynamicSentenceMemory()

seed_text = """
The player stole Mitch's shoes.
Mitch keeps spare boots under his bed.
A bartender saw muddy prints near the back door.
Mitch suspects the player but has no proof.
The town square fountain is cracked and leaking.
"""

memory.add_text(seed_text)

# Runtime insertion: add new facts during play
memory.add_sentences([
    "Mitch found one stolen boot beside the docks.",
    "A guard reported the player carrying footwear at dawn.",
])

print(f"Indexed sentences: {len(memory.sentences)}")


Indexed sentences: 7


In [5]:
query_paragraph = """
Mitch complains that his footwear is missing after a theft,
and he says the player may have taken his boots near the harbor.
"""

top_n = 4
hits = memory.search(query_paragraph, top_n=top_n)

for rank, hit in enumerate(hits, start=1):
    print(f"{rank}. score={hit.score:.4f} | {hit.sentence}")


1. score=0.7182 | The player stole Mitch's shoes.
2. score=0.6826 | Mitch found one stolen boot beside the docks.
3. score=0.6034 | Mitch keeps spare boots under his bed.
4. score=0.5353 | Mitch suspects the player but has no proof.


The relevance behavior comes from the embedding model + vectors in the index, not manual semantic rules in notebook code.